In [ ]:
using PyPlot
using AlfvenDetectors
using BSON
using Flux
using ValueHistories
using StatsBase
using GaussianMixtures
using Random
include("../../experiments/eval_utils.jl")

In [ ]:
datapath = "/home/vit/vyzkum/alfven/cdb_data/uprobe_data"
shots = joinpath.(datapath, readdir(datapath))
shotnos, labels, tstarts, fstarts = AlfvenDetectors.labeled_patches()
patchsize = 128
readfun = AlfvenDetectors.readnormlogupsd
cmap = "plasma"

In [ ]:
patchpath = "/home/vit/vyzkum/alfven/experiments/conv/labeled_patches/$patchsize"
batchsize = 10
data = map(x->AlfvenDetectors.get_patch_from_csv(patchpath, x[1], x[2], x[3], x[4]), 
        zip(shotnos, labels, tstarts, fstarts))
data = cat(map(x->reshape(x,size(x)...,1,1),data)...,dims=4)
println(size(data))

In [ ]:
alfvendata = data[:,:,:,labels.==1]
noalfvendata = data[:,:,:,labels.==0];

In [ ]:
modelpath = "/home/vit/vyzkum/alfven/experiments/conv/uprobe/benchmark-runs"
model_file = joinpath(modelpath, "ConvAE_xdim-(128, 128, 1)_ldim-32_nlayers-2_kernelsize-3_channels-[8, 16]_scaling-2_outbatchnorm-true_batchnorm-true_usegpu-true_memoryefficient-true_batchsize-128_opt-RMSProp_eta-0.001_nshots-10_nepochs-1000_noalfven-false_2019-04-13T10:31:22.045.bson")

In [ ]:
params = parse_params(model_file)
model_data = BSON.load(model_file)
history = model_data[:history]
if get(params, :batchnorm, false)
    model = Flux.testmode!(model_data[:model])
else
    model = model_data[:model]
end
print(pretty_params(params))

In [ ]:
z = model.encoder(data).data;
zT = Array(z')

Do train/test split.

In [ ]:
Random.seed!(12345)
N = size(z,2)
rand_inds = sample(1:N,N,replace=false)
train_inds = rand_inds[1:Int(N/2)]
test_inds = rand_inds[Int(N/2)+1:end]
z_train = z[:,train_inds]
z_test = z[:,test_inds]
labels_train = labels[train_inds]
labels_test = labels[test_inds]
zT_train = Array(z_train')
zT_test = Array(z_test')

In [ ]:
#kind = :full
kind = :diag
gmm_model = GaussianMixtures.GMM(5, zT_train, kind=kind)

In [ ]:
println(gmm_model.w)

### Compute the anomaly score

First compute the mean/max/median log-likelihood of all the datapoints relative to all components. Use this N-component long vector as a base. Then, for a new point, compute its log-likelihood in the components and use the negative value of the MSE between this vector and the base as the anomaly score.

In [ ]:
score(model, x) = llpg(model, x)
maxscore(model,x) = maximum(score(model,x),dims=1)
meanscore(model,x) = mean(score(model,x),dims=1)
medianscore(model,x) = StatsBase.median(score(model,x),dims=1)
max_inds(model, x) = map(x->x[2],argmax(score(model, x),dims=2))    

In [ ]:
ma = meanscore(gmm_model, zT_train[labels_train.==1,:])
println(ma)
mn = meanscore(gmm_model, zT_train[labels_train.==0,:])
println(mn)

In [ ]:
mamax = maxscore(gmm_model, zT_train[labels_train.==1,:])
println(mamax)
mnmax = maxscore(gmm_model, zT_train[labels_train.==0,:])
println(mnmax)

In [ ]:
mamed = medianscore(gmm_model, zT_train[labels_train.==1,:])
println(mamed)
mnmed = medianscore(gmm_model, zT_train[labels_train.==0,:])
println(mnmed)

In [ ]:
i = 7
println(labels_test[i])
lls = score(gmm_model, zT_test[i:i,:])
println(lls)
s = sum((lls - ma).^2)
println(s)

In [ ]:
lls_test = score(gmm_model, zT_test);
sm = vec(mean((lls_test.-ma).^2,dims=2)/size(ma,2));
smax = vec(mean((lls_test.-mamax).^2,dims=2)/size(mamax,2));
smed = vec(mean((lls_test.-mamed).^2,dims=2)/size(mamed,2));

In [ ]:
sortinds = sortperm(sm)
for i in sortinds
    println(labels_test[i], " ", sm[i])
end

In [ ]:
using EvalCurves

In [ ]:
roc = EvalCurves.roccurve(-smed, labels_test)
auroc = EvalCurves.auc(roc...)

In [ ]:
title("$auroc")
plot(roc...)
xlim([0,1])
ylim([0,1])

Obviously, this is not very good.

### Try something else

Now, lets figure out which component is most likely for the labeled patches to appear. Then, take this component as the base. The anomaly score is then the negative of loglikelihood of teh sample in this component. Or maybe take the K most frequently appearing components?

In [ ]:
alfven_inds = max_inds(gmm_model, zT_train[labels_train.==1,:]);
noalfven_inds = max_inds(gmm_model, zT_train[labels_train.==0,:]);

In [ ]:
plt.hist(alfven_inds)

In [ ]:
plt.hist(noalfven_inds)

In [ ]:
alfven_inds_test = max_inds(gmm_model, zT_test[labels_test.==1,:]);
noalfven_inds_test = max_inds(gmm_model, zT_test[labels_test.==0,:]);

In [ ]:
plt.hist(alfven_inds_test)

In [ ]:
plt.hist(noalfven_inds_test)

In [ ]:
function component_frequency(model, x)
    maxinds = vec(max_inds(model, x))
    cmap = countmap(maxinds)
    is = collect(keys(cmap))
    counts = collect(values(cmap))
    isort = sortperm(counts,rev=true)
    return is[isort], counts[isort]
end

In [ ]:
cinds, ccounts = component_frequency(gmm_model, zT_train[labels_train.==1,:])

In [ ]:
llhs = score(gmm_model, zT_test)
as = llhs[:,cinds[1]]

In [ ]:
max_inds_x = vec(max_inds(gmm_model, zT_test))

In [ ]:
as = Int.(max_inds_x .== cinds[1])

In [ ]:
roc = EvalCurves.roccurve(as, labels_test)
auroc = EvalCurves.auc(roc...)

In [ ]:
plot(roc...)
title("$auroc")

In [ ]:
bestinds = sortperm(-as)
labels_test[bestinds]

Try SVAE, kNN on the latent space? Try AAEs, Vamps, Waserstein...

In [ ]:
for i in bestinds
    figure()
    pcolormesh(data[:,:,1,test_inds[i]],cmap=cmap)
    title("$(labels_test[i])")
end